# Gradient Boosting 

In [1]:
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
# load basic libraries
import pandas as pd
import numpy as np
import mne
from pathlib import Path
import pickle
import time

from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, cross_val_score

%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt

# set directories
# %cd D:/Programy/Anaconda3/Projects/EEG ML project # working directory
%cd D:
pkls = './Pickles/' # objects & variables

D:\Bartek\JupyterLab Projects\ML EEG project


In [3]:
# load split sets, eeg only
with open(pkls +'xy_train.pkl', 'rb') as handle:
    x_train = pickle.load(handle)
    y_train = pickle.load(handle)
y_train.shape
x_train.shape

(6040, 94)

### Early stopping with Grid search

In [14]:
cv = 10
n_gbes = []
time_gbes = []

param_grid = {'max_depth': [2, 3, 4], 
              'learning_rate': [0.1, 0.5, 1.5]} 

scorers = {'accuracy': make_scorer(accuracy_score),
           'f1'      : make_scorer(f1_score, average = 'weighted')}

grid = GridSearchCV(GradientBoostingClassifier(validation_fraction = 0.2,
                                               n_iter_no_change=10, 
                                               tol=0.01,
                                               random_state=42), 
                    param_grid, refit='accuracy', verbose=1, 
                    scoring = scorers, return_train_score=True, cv = cv, n_jobs = 6)

start = time.time()

grid.fit(x_train, y_train.values.ravel())
best_gbc = grid.best_estimator_
best_gbc

time_gbes.append(time.time() - start)
n_gbes.append(gbes.n_estimators_)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done  90 out of  90 | elapsed:  3.4min finished


In [15]:
time_gbes
n_gbes
best_gbc

In [18]:
# evaluate and save
best_gbc_res = round(pd.DataFrame(index = ['gbc'], data  = 
          {'acc mean':[grid.cv_results_['mean_test_accuracy'][grid.best_index_]],
           'acc 2sd': [2*grid.cv_results_['std_test_accuracy'][grid.best_index_]],
           'f1 mean': [grid.cv_results_['mean_test_f1'][grid.best_index_]],
           'f1 2sd' : [2*grid.cv_results_['std_test_f1'][grid.best_index_]]}),4)
best_gbc_res

,acc mean,acc 2sd,f1 mean,f1 2sd
gbc,0.697,0.0344,0.6954,0.0355


In [19]:
# train set score for comparison
best_gbc.fit(x_train, y_train.values.ravel())
print("training acc mean: {:.4f}".format(
    best_gbc.score(x_train, y_train)))
print("training f1  mean: {:.4f}".format(
    f1_score(y_train, best_gbc.predict(x_train), average = 'weighted')))

training acc mean: 0.9017
training f1  mean: 0.9015


In [7]:
# save best
with open('./Pickles/best_gbc.pkl', 'wb') as handle:
    pickle.dump(best_gbc, handle)
    pickle.dump(best_gbc_res, handle)

In [5]:
with open(pkls + 'best_gbc.pkl', 'rb') as handle:
    best_gbc = pickle.load(handle)
    best_gbc_res = pickle.load(handle)